## healthcare_data_manipulation.ipnyb

This program extracts synthetic healthcare data from a database stored on SQL Server and Snowflake. It then manipulates the data in various ways.

Written by Stephen Lew

In [18]:
import os
import pyodbc
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sqlalchemy import create_engine

### Load environment variables from .env file

In [19]:
load_dotenv()

True

### Extract data from database stored on SQL Server

Use a **user-defined function** and an **f-string** to make the process easier to understand.

In [20]:
def mssql_engine(server, database):
    engine = create_engine(
        f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes"
    )
    return engine

engine1 = mssql_engine(server = "localhost\\SQLEXPRESS",
                       database = "SyntheticHealthcareData")

Synthetic healthcare claims data

Source: https://www.kaggle.com/datasets/abuthahir1998/synthetic-healthcare-claims-dataset/data

In [21]:
df_claim_data = pd.read_sql("SELECT * FROM claim_data", engine1)
df_claim_data.head(5)

,claim_id,provider_id,patient_id,date_of_service,billed_amount,procedure_code,diagnosis_code,allowed_amount,paid_amount,insurance_type,claim_status,reason_code,followup_required,ar_status,outcome
0,0HO1FSN4AP,0126528997,7936697103,2024-08-07,304.0,99231,A02.1,218.0,203.0,Self-Pay,Paid,Incorrect billing information,Yes,Pending,Partially Paid
1,9U86CI2P5A,6986719948,1547160031,2024-06-21,348.0,99213,T40.905A,216.0,206.0,Medicare,Paid,Pre-existing condition,Yes,Open,Denied
2,1QEU1AIDAU,1355108115,2611585318,2024-07-04,235.0,99213,A00.1,148.0,119.0,Commercial,Under Review,Duplicate claim,No,Denied,Denied
3,WH7XDS8CEO,9991055906,7167948632,2024-05-26,112.0,99215,A18.6,79.0,69.0,Medicare,Denied,Authorization not obtained,No,Partially Paid,Denied
4,M6OJEZ8KGI,7382167012,2140226267,2024-07-16,406.0,99238,A17.9,320.0,259.0,Medicare,Denied,Authorization not obtained,No,On Hold,Denied


ICD 10 code descriptions

Source: https://www.kaggle.com/datasets/mrhell/icd10cm-codeset-2023

In [22]:
df_icd_codes = pd.read_sql("SELECT * FROM icd_codes", engine1)
df_icd_codes.head(5)

,icd_code,description
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,"Cholera, unspecified"
3,A01.00,"Typhoid fever, unspecified"
4,A01.01,Typhoid meningitis


### Extract data from database stored on Snowflake

Use a **user-defined function** and an **f-string** to make the process easier to understand.

In [23]:
def snowflake_engine(uid, pwd, account, database, schema):
    engine = create_engine(
        f"snowflake://{uid}:{pwd}@{account}/{database}/{schema}?warehouse=COMPUTE_WH&role=ACCOUNTADMIN"
    )

    return engine

engine2 = snowflake_engine(uid = os.getenv("SNOWFLAKE_USER"),
                           pwd = os.getenv("SNOWFLAKE_PASSWORD"),
                           account = os.getenv("SNOWFLAKE_ACCOUNT"),
                           database = "SYNTHETICHEALTHCAREDATA",
                           schema = "PUBLIC")

In [24]:
del df_claim_data
df_claim_data = pd.read_sql("SELECT * FROM claim_data", engine2)
df_claim_data.head(5)

,claim_id,provider_id,patient_id,date_of_service,billed_amount,procedure_code,diagnosis_code,allowed_amount,paid_amount,insurance_type,claim_status,reason_code,followup_required,ar_status,outcome
0,0HO1FSN4AP,0126528997,7936697103,2024-08-07,304.0,99231,A02.1,218.0,203.0,Self-Pay,Paid,Incorrect billing information,Yes,Pending,Partially Paid
1,9U86CI2P5A,6986719948,1547160031,2024-06-21,348.0,99213,T40.905A,216.0,206.0,Medicare,Paid,Pre-existing condition,Yes,Open,Denied
2,1QEU1AIDAU,1355108115,2611585318,2024-07-04,235.0,99213,A00.1,148.0,119.0,Commercial,Under Review,Duplicate claim,No,Denied,Denied
3,WH7XDS8CEO,9991055906,7167948632,2024-05-26,112.0,99215,A18.6,79.0,69.0,Medicare,Denied,Authorization not obtained,No,Partially Paid,Denied
4,M6OJEZ8KGI,7382167012,2140226267,2024-07-16,406.0,99238,A17.9,320.0,259.0,Medicare,Denied,Authorization not obtained,No,On Hold,Denied


In [25]:
del df_icd_codes
df_icd_codes = pd.read_sql("SELECT * FROM icd_codes", engine2)
df_icd_codes.head(5)

,icd_code,description
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol..."
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor"
2,A00.9,"Cholera, unspecified"
3,A01.00,"Typhoid fever, unspecified"
4,A01.01,Typhoid meningitis


### Join tables and rename field

Bring in descriptions of ICD-10 codes into the claims data.

Rename the new field from "description" to "diagnosis description".

In [26]:
df_claim_data = (df_claim_data.merge(df_icd_codes, left_on = "diagnosis_code", right_on = "icd_code", how = "left")
                 .rename(columns = {"description": "diagnosis_description"})
                 .drop("icd_code", axis = 1))
df_claim_data.head(5)

,claim_id,provider_id,patient_id,date_of_service,billed_amount,procedure_code,diagnosis_code,allowed_amount,paid_amount,insurance_type,claim_status,reason_code,followup_required,ar_status,outcome,diagnosis_description
0,0HO1FSN4AP,0126528997,7936697103,2024-08-07,304.0,99231,A02.1,218.0,203.0,Self-Pay,Paid,Incorrect billing information,Yes,Pending,Partially Paid,Salmonella sepsis
1,9U86CI2P5A,6986719948,1547160031,2024-06-21,348.0,99213,T40.905A,216.0,206.0,Medicare,Paid,Pre-existing condition,Yes,Open,Denied,Adverse effect of unspecified psychodysleptics...
2,1QEU1AIDAU,1355108115,2611585318,2024-07-04,235.0,99213,A00.1,148.0,119.0,Commercial,Under Review,Duplicate claim,No,Denied,Denied,"Cholera due to Vibrio cholerae 01, biovar eltor"
3,WH7XDS8CEO,9991055906,7167948632,2024-05-26,112.0,99215,A18.6,79.0,69.0,Medicare,Denied,Authorization not obtained,No,Partially Paid,Denied,Tuberculosis of (inner) (middle) ear
4,M6OJEZ8KGI,7382167012,2140226267,2024-07-16,406.0,99238,A17.9,320.0,259.0,Medicare,Denied,Authorization not obtained,No,On Hold,Denied,"Tuberculosis of nervous system, unspecified"


### Create variable and summarise data

Create a new dataframe that has, for each insurance type, the rate at which claims are denied.

In [27]:
df_claim_data["denial_rate"] = np.where(df_claim_data["claim_status"] == "Denied", 100, 0)
df_denial_rate = (df_claim_data.groupby("insurance_type")["denial_rate"].mean()
                  .reset_index())
df_denial_rate


,insurance_type,denial_rate
0,Commercial,34.362934
1,Medicaid,28.957529
2,Medicare,36.051502
3,Self-Pay,32.128514


### Filter data

Subset the data to only records with a diagnosis code of A05.4 (Foodborne Bacillus cereus intoxication).

In [28]:
df_filter = df_claim_data.query("diagnosis_code == 'A05.4'")
df_filter.head(5)

,claim_id,provider_id,patient_id,date_of_service,billed_amount,procedure_code,diagnosis_code,allowed_amount,paid_amount,insurance_type,claim_status,reason_code,followup_required,ar_status,outcome,diagnosis_description,denial_rate
8,TLU7MTJ63J,6675789567,1861061750,2024-07-16,126.0,99238,A05.4,109.0,108.0,Medicare,Denied,Authorization not obtained,No,Closed,Partially Paid,Foodborne Bacillus cereus intoxication,100
59,Y4I4JQPVK4,8925429111,9342177346,2024-07-11,267.0,99215,A05.4,214.0,171.0,Self-Pay,Under Review,Authorization not obtained,Yes,Open,Paid,Foodborne Bacillus cereus intoxication,0
213,TD0EG8I93R,6744823555,6194960851,2024-08-17,349.0,99231,A05.4,273.0,267.0,Commercial,Paid,Service not covered,No,Partially Paid,Partially Paid,Foodborne Bacillus cereus intoxication,0
253,RS1YY0WY24,5431725002,9176601831,2024-05-28,111.0,99238,A05.4,81.0,76.0,Self-Pay,Paid,Lack of medical necessity,Yes,Closed,Denied,Foodborne Bacillus cereus intoxication,0
263,BADGAN63O1,0456145390,9008471630,2024-07-25,172.0,99222,A05.4,147.0,127.0,Medicare,Under Review,Incorrect billing information,Yes,Open,Paid,Foodborne Bacillus cereus intoxication,0


### Sort data

Sort data by date of service then claim ID.

In [29]:
(df_claim_data.sort_values(["date_of_service", "claim_id"])
 .head(5))

,claim_id,provider_id,patient_id,date_of_service,billed_amount,procedure_code,diagnosis_code,allowed_amount,paid_amount,insurance_type,claim_status,reason_code,followup_required,ar_status,outcome,diagnosis_description,denial_rate
235,4AECRPPSTL,6034872297,1183596686,2024-05-01,214.0,99213,M72.8,179.0,178.0,Commercial,Under Review,Duplicate claim,No,Partially Paid,Partially Paid,Other fibroblastic disorders,0
403,8H6FAKD1OH,1109442085,5127757508,2024-05-01,244.0,99238,A04.8,168.0,150.0,Medicaid,Under Review,Duplicate claim,Yes,Pending,Paid,Other specified bacterial intestinal infections,0
267,AFCVL9N05Z,0369668919,3458366761,2024-05-01,108.0,99222,A05.0,66.0,58.0,Medicaid,Paid,Missing documentation,Yes,Closed,Partially Paid,Foodborne staphylococcal intoxication,0
919,WR6QACIBSK,6636473801,6871324728,2024-05-01,252.0,99213,M93.259,222.0,183.0,Commercial,Paid,Duplicate claim,Yes,Denied,Partially Paid,"Osteochondritis dissecans, unspecified hip",0
174,0APP6HH8P5,5577749071,8116288647,2024-05-02,224.0,99221,A05.3,158.0,133.0,Commercial,Denied,Lack of medical necessity,Yes,Open,Paid,Foodborne Vibrio parahaemolyticus intoxication,100


### Keep select fields

Keep only the fields for claim ID, procedure code, and diagnosis code.

In [30]:
df_select = df_claim_data[["claim_id", "procedure_code", "diagnosis_code"]]
df_select.head(5)

,claim_id,procedure_code,diagnosis_code
0,0HO1FSN4AP,99231,A02.1
1,9U86CI2P5A,99213,T40.905A
2,1QEU1AIDAU,99213,A00.1
3,WH7XDS8CEO,99215,A18.6
4,M6OJEZ8KGI,99238,A17.9


### Perform the same operation on multiple columns

For each field insurance_type, claim_status, reason_code, followup_required, ar_status, and outcome, create a new field with the value in uppercase.

Use a **for loop** with **tuple unpacking** and **f-strings**.

In [31]:
list_chr_vars = [
    ("insurance_type", "insurance_type_upcase"),
    ("claim_status", "claim_status_upcase"),
    ("reason_code", "reason_code_upcase"),
    ("followup_required", "followup_required_upcase"),
    ("ar_status", "ar_status_upcase"),
    ("outcome", "outcome_upcase")
]
for mcase, ucase in list_chr_vars:
    df_claim_data[f"{ucase}"] = df_claim_data[f"{mcase}"].str.upper()
df_claim_data.head(5)

,claim_id,provider_id,patient_id,date_of_service,billed_amount,procedure_code,diagnosis_code,allowed_amount,paid_amount,insurance_type,...,ar_status,outcome,diagnosis_description,denial_rate,insurance_type_upcase,claim_status_upcase,reason_code_upcase,followup_required_upcase,ar_status_upcase,outcome_upcase
0,0HO1FSN4AP,0126528997,7936697103,2024-08-07,304.0,99231,A02.1,218.0,203.0,Self-Pay,...,Pending,Partially Paid,Salmonella sepsis,0,SELF-PAY,PAID,INCORRECT BILLING INFORMATION,YES,PENDING,PARTIALLY PAID
1,9U86CI2P5A,6986719948,1547160031,2024-06-21,348.0,99213,T40.905A,216.0,206.0,Medicare,...,Open,Denied,Adverse effect of unspecified psychodysleptics...,0,MEDICARE,PAID,PRE-EXISTING CONDITION,YES,OPEN,DENIED
2,1QEU1AIDAU,1355108115,2611585318,2024-07-04,235.0,99213,A00.1,148.0,119.0,Commercial,...,Denied,Denied,"Cholera due to Vibrio cholerae 01, biovar eltor",0,COMMERCIAL,UNDER REVIEW,DUPLICATE CLAIM,NO,DENIED,DENIED
3,WH7XDS8CEO,9991055906,7167948632,2024-05-26,112.0,99215,A18.6,79.0,69.0,Medicare,...,Partially Paid,Denied,Tuberculosis of (inner) (middle) ear,100,MEDICARE,DENIED,AUTHORIZATION NOT OBTAINED,NO,PARTIALLY PAID,DENIED
4,M6OJEZ8KGI,7382167012,2140226267,2024-07-16,406.0,99238,A17.9,320.0,259.0,Medicare,...,On Hold,Denied,"Tuberculosis of nervous system, unspecified",100,MEDICARE,DENIED,AUTHORIZATION NOT OBTAINED,NO,ON HOLD,DENIED
